# **1. Obtener el valor actual de la UF desde una API**

Una opción fiable es la API pública [Mindicador.cl](https://mindicador.cl/), que entrega indicadores económicos en formato JSON, incluyendo el [valor de la UF](https://mindicador.cl/api/uf).

In [118]:
import requests

def obtener_valor_uf():
    url = "https://www.mindicador.cl/api/uf/"
    respuesta = requests.get(url)
    data = respuesta.json()
    return data['serie'][0]['valor']  # valor más reciente de la UF

valor_uf = obtener_valor_uf()
print(f"Valor UF actual: {valor_uf:.2f} CLP")

Valor UF actual: 39167.40 CLP


# **2. Estructura para el calculo de amortización**

**¿Qué es la amortización y por qué es clave en finanzas?**

> _"La amortización es la disminución gradual de la deuda -el valor- gracias al pago de las cuotas pactadas -el tiempo- con el banco. Generalmente, cuando se habla de la amortización de un pasivo se tiene en cuenta únicamente al valor del capital, sin incluir el de los [intereses](https://www.santander.com/es/stories/todo-lo-que-necesitas-saber-sobre-los-intereses) que se generan."_

**Santander. (s. f.). ¿Qué es la amortización y por qué es clave en finanzas? https://www.santander.com/es/stories/amortizacion**

## **a) Librerías necesarias**

In [119]:
import requests
import pandas as pd
import numpy as np

## **b) Función para generar tabla de amortización**

In [120]:
def tabla_amortizacion(prestamo, tasa_anual, plazo_anos):
    tasa_mensual = (tasa_anual / 100) / 12
    n_meses = plazo_anos * 12
    cuota = (prestamo * tasa_mensual) / (1 - (1 + tasa_mensual) ** -n_meses)
    saldo = prestamo
    filas = []
    for mes in range(1, n_meses + 1):
        interes = saldo * tasa_mensual
        amort = cuota - interes
        saldo -= amort
        filas.append([mes, cuota, interes, amort, max(saldo, 0)])
    df = pd.DataFrame(filas, columns=["Mes", "Cuota_UF", "Interés_UF", "Amortización_UF", "Saldo_UF"])
    return df


## **c) Celdas para ingreso de variables por el usuario**

In [121]:
valor_uf_clp = obtener_valor_uf()
print(f"Valor UF actual en CLP: {valor_uf_clp:.2f}")

valor_propiedad = 3000 # Valor propiedad (UF)
porc_financ = 80       # Porcentaje financiamiento (ej. 80 para 80%)
tasa_anual = 5.5       # Tasa anual (ej. 5.5)
plazo_anos = 25        # Plazo en años

prestamo_uf = valor_propiedad * (porc_financ / 100)

print(f'Prestamo en UF {prestamo_uf}')


Valor UF actual en CLP: 39167.40
Prestamo en UF 2400.0


In [122]:
df_amort = tabla_amortizacion(prestamo_uf, tasa_anual, plazo_anos)
df_amort.head()

,Mes,Cuota_UF,Interés_UF,Amortización_UF,Saldo_UF
0,1,15,11,4,"2,396"
1,2,15,11,4,"2,393"
2,3,15,11,4,"2,389"
3,4,15,11,4,"2,385"
4,5,15,11,4,"2,381"


## **d) Mostrar resultados**

In [123]:
# Resumen anual en UF
df_amort['Año'] = ((df_amort['Mes'] - 1) // 12) + 1
df_res_ano = df_amort.groupby('Año')[['Cuota_UF', 'Interés_UF', 'Amortización_UF']].sum()

df_res_ano.head()

,Cuota_UF,Interés_UF,Amortización_UF
Año,,,
1,177,131,46
2,177,128,49
3,177,126,51
4,177,123,54
5,177,120,57


In [124]:
df_amort_clp = df_amort.copy()
df_amort_clp[['Cuota_CLP', 'Interés_CLP', 'Amortización_CLP', 'Saldo_CLP']] = df_amort_clp[['Cuota_UF','Interés_UF','Amortización_UF','Saldo_UF']]*valor_uf_clp

df_amort_clp.head()

,Mes,Cuota_UF,Interés_UF,Amortización_UF,Saldo_UF,Año,Cuota_CLP,Interés_CLP,Amortización_CLP,Saldo_CLP
0,1,15,11,4,"2,396",1,"577,253","430,841","146,412","93,855,348"
1,2,15,11,4,"2,393",1,"577,253","430,170","147,083","93,708,266"
2,3,15,11,4,"2,389",1,"577,253","429,496","147,757","93,560,509"
3,4,15,11,4,"2,385",1,"577,253","428,819","148,434","93,412,075"
4,5,15,11,4,"2,381",1,"577,253","428,139","149,114","93,262,960"


In [125]:
# Redondeando CLP solo para mostrar (sin alterar datos)
pd.options.display.float_format = '{:,.2f}'.format
df_amort_clp.head()

,Mes,Cuota_UF,Interés_UF,Amortización_UF,Saldo_UF,Año,Cuota_CLP,Interés_CLP,Amortización_CLP,Saldo_CLP
0,1,14.74,11.00,3.74,"2,396.26",1,"577,253.05","430,841.40","146,411.65","93,855,348.35"
1,2,14.74,10.98,3.76,"2,392.51",1,"577,253.05","430,170.35","147,082.70","93,708,265.65"
2,3,14.74,10.97,3.77,"2,388.73",1,"577,253.05","429,496.22","147,756.83","93,560,508.81"
3,4,14.74,10.95,3.79,"2,384.94",1,"577,253.05","428,819.00","148,434.05","93,412,074.76"
4,5,14.74,10.93,3.81,"2,381.14",1,"577,253.05","428,138.68","149,114.37","93,262,960.39"


In [126]:
df_amort_clp.tail()

,Mes,Cuota_UF,Interés_UF,Amortización_UF,Saldo_UF,Año,Cuota_CLP,Interés_CLP,Amortización_CLP,Saldo_CLP
295,296,14.74,0.33,14.40,58.28,25,"577,253.05","13,048.75","564,204.30","2,282,795.37"
296,297,14.74,0.27,14.47,43.81,25,"577,253.05","10,462.81","566,790.24","1,716,005.13"
297,298,14.74,0.20,14.54,29.27,25,"577,253.05","7,865.02","569,388.03","1,146,617.10"
298,299,14.74,0.13,14.60,14.67,25,"577,253.05","5,255.33","571,997.72","574,619.38"
299,300,14.74,0.07,14.67,0.00,25,"577,253.05","2,633.67","574,619.38",0.00


# **3. Calculo proyectado con inflación constante**

In [ ]:
inflacion_anual = 0.03  # 3% inflación anual proyectada
valor_uf_inicial = valor_uf_clp  # uf actual en pesos

df_amort_proyectado = df_amort.copy()
df_amort_proyectado['Valor_UF_proyectado'] = [valor_uf_inicial * (1 + inflacion_anual) ** (mes/12) for mes in df_amort_proyectado['Mes']]

In [128]:
df_amort_clp_proyectado = df_amort_proyectado.copy()
df_amort_clp_proyectado[['Cuota_CLP', 'Cuota_CLP_proyectada', 'Interés_CLP', 'Amortización_CLP', 'Saldo_CLP']] = df_amort_clp_proyectado[['Cuota_UF', 'Valor_UF_proyectado', 'Interés_UF','Amortización_UF','Saldo_UF']] * valor_uf_clp

# Cuota en pesos ajustada mes a mes según inflación.
df_amort_clp_proyectado['Cuota_CLP_proyectada'] = df_amort_proyectado['Cuota_UF'] * df_amort_proyectado['Valor_UF_proyectado']
df_amort_clp_proyectado['Saldo_CLP_proyectado'] = df_amort_proyectado['Saldo_UF'] * df_amort_proyectado['Valor_UF_proyectado']

df_amort_clp_proyectado.head()

,Mes,Cuota_UF,Interés_UF,Amortización_UF,Saldo_UF,Año,Valor_UF_proyectado,Cuota_CLP,Cuota_CLP_proyectada,Interés_CLP,Amortización_CLP,Saldo_CLP,Saldo_CLP_proyectado
0,1,14.74,11.00,3.74,"2,396.26",1,"39,264.00","577,253.05","578,676.71","430,841.40","146,411.65","93,855,348.35","94,086,820.96"
1,2,14.74,10.98,3.76,"2,392.51",1,"39,360.83","577,253.05","580,103.89","430,170.35","147,082.70","93,708,265.65","94,171,055.35"
2,3,14.74,10.97,3.77,"2,388.73",1,"39,457.91","577,253.05","581,534.58","429,496.22","147,756.83","93,560,508.81","94,254,453.82"
3,4,14.74,10.95,3.79,"2,384.94",1,"39,555.22","577,253.05","582,968.80","428,819.00","148,434.05","93,412,074.76","94,337,006.94"
4,5,14.74,10.93,3.81,"2,381.14",1,"39,652.78","577,253.05","584,406.56","428,138.68","149,114.37","93,262,960.39","94,418,705.20"


In [129]:
df_amort_clp_proyectado.tail()

,Mes,Cuota_UF,Interés_UF,Amortización_UF,Saldo_UF,Año,Valor_UF_proyectado,Cuota_CLP,Cuota_CLP_proyectada,Interés_CLP,Amortización_CLP,Saldo_CLP,Saldo_CLP_proyectado
295,296,14.74,0.33,14.40,58.28,25,"81,203.79","577,253.05","1,196,789.53","13,048.75","564,204.30","2,282,795.37","4,732,804.06"
296,297,14.74,0.27,14.47,43.81,25,"81,404.06","577,253.05","1,199,741.13","10,462.81","566,790.24","1,716,005.13","3,566,480.82"
297,298,14.74,0.20,14.54,29.27,25,"81,604.82","577,253.05","1,202,700.02","7,865.02","569,388.03","1,146,617.10","2,388,963.39"
298,299,14.74,0.13,14.60,14.67,25,"81,806.08","577,253.05","1,205,666.20","5,255.33","571,997.72","574,619.38","1,200,165.44"
299,300,14.74,0.07,14.67,0.00,25,"82,007.84","577,253.05","1,208,639.70","2,633.67","574,619.38",0.00,0.00


## **a) Limpiar tabla**

In [130]:
df_amort_clp_proyectado.drop(columns=['Cuota_UF', 'Interés_UF', 'Amortización_UF', 'Saldo_UF', 'Cuota_CLP', 'Interés_CLP', 'Amortización_CLP', 'Saldo_CLP'], axis=1, inplace=True)

## **b) Ver datos CLP proyectados**

In [131]:
df_amort_clp_proyectado.head()

,Mes,Año,Valor_UF_proyectado,Cuota_CLP_proyectada,Saldo_CLP_proyectado
0,1,1,"39,264.00","578,676.71","94,086,820.96"
1,2,1,"39,360.83","580,103.89","94,171,055.35"
2,3,1,"39,457.91","581,534.58","94,254,453.82"
3,4,1,"39,555.22","582,968.80","94,337,006.94"
4,5,1,"39,652.78","584,406.56","94,418,705.20"


In [ ]:
df_amort_clp_proyectado.tail()

,Mes,Año,Valor_UF_proyectado,Cuota_CLP_proyectada,Saldo_CLP_proyectado
295,296,25,"81,203.79","1,196,789.53","4,732,804.06"
296,297,25,"81,404.06","1,199,741.13","3,566,480.82"
297,298,25,"81,604.82","1,202,700.02","2,388,963.39"
298,299,25,"81,806.08","1,205,666.20","1,200,165.44"
299,300,25,"82,007.84","1,208,639.70",0.00
